<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План
    1. Введение
    2. Описание данных
    3. Предобработка данных
    4. Обучение и проверка моделей
    5. Заключение

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [2]:
!head -1 stackoverflow_10mln.tsv

"head" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l stackoverflow_10mln.tsv

Wall time: 26 ms


"wc" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [3]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

In [4]:
input_filename = 'stackoverflow_10mln.tsv'
output_filename  = 'stackoverflow.vw'
output_file = open(output_filename, 'a')

#теги классов, на которые мы будем классифицировать весь текст
target_tags = ['javascript', 'java', 'python', 'ruby', 'php', 'c++', 'c#', 'go', 'scala', 'swift']

with open(input_filename) as file:
	for line in tqdm(file, total = 10000000):
		line = line.strip()        
		if (line.count('\t') == 0) or (line.count('\t') > 1):
			continue
		else:		             
			splited_line = line.split('\t')
			text = splited_line[0]
			tags = splited_line[1]
			tags = tags.replace('\n', '')
			tags = tags.split(' ')
			tags_counter = 0
			new_tags = []           
			for tag in tags:
				if (tag in target_tags):
					new_tags.append(tag)
				tags = new_tags
			for tag in target_tags:
				if (tag in tags):
					tags_counter = tags_counter + 1
			if (tags_counter != 1):
				continue
			else:
				text = text.replace('?','')
				text = text.replace(':','')
				text = text.replace('|','')
				new_line = str(int(target_tags.index(tags[0])) + 1) + ' | ' + str(text) + '\n'                
				output_file.write(new_line)
output_file.close()

100%|██████████| 10000000/10000000 [06:34<00:00, 25342.79it/s]


Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [5]:
!split -l 1463018 stackoverflow.vw

"split" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [6]:
!cut -c1 stackoverflow_valid.vw > stackoverflow_valid_labels.txt
!cut -c1 stackoverflow_train.vw > stackoverflow_valid_train.txt

"cut" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.
"cut" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [ ]:
%%time
best_score = 0

for ngram in (1,2,3):
    for passes in (1,3,5):
        !vw --cache --oaa 10 --random_seed 17 --bit_precision 28 --loss_function=hinge --ngram {ngram} --passes {passes} \
        -d stackoverflow_train.vw -f model.vw
        
        !vw -i model.vw -t -d stackoverflow_valid.vw -p stackoverflow_valid_pred.txt --quiet
        
        with open('stackoverflow_valid_pred.txt') as pred_file:
            valid_pred = [float(label) for label in pred_file.readlines()]
            
        with open('stackoverflow_valid_labels.txt') as labels_file:
            valid_labels = [float(label) for label in labels_file.readlines()]   
        
        current_score = accuracy_score(valid_labels, valid_pred)
        print ("For ngram:{} and pass:{} - score:{}".format(ngram, passes, current_score))
        if (current_score > best_score):
            best_score = current_score
            best_ngram = ngram
            best_passes = passes

Generating 1-grams for all namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.750000 0.750000           16           16.0        7        7      209
0.781250 0.812500           32           32.0        7        2      174
0.765625 0.750000           64           64.0        3        3      204
0.648438 0.531250          128          128.0        1        5       29
0.609375 0.570313          

For ngram:1 and pass:1 - score:0.8964619710762274


Generating 1-grams for all namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.750000 0.750000           16           16.0        2        7      159
0.750000 0.750000           32           32.0        1        7      404
0.703125 0.656250           64           64.0        7        7      103
0.617188 0.531250          128          128.0        5        5      276
0.5

For ngram:1 and pass:3 - score:0.8976424076805617


Generating 1-grams for all namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.750000 0.750000           16           16.0        2        7      159
0.750000 0.750000           32           32.0        1        7      404
0.703125 0.656250           64           64.0        7        7      103
0.617188 0.531250          128          128.0        5        5      276
0.5

...

For ngram:2 and pass:5 - score:0.910154215464198


Generating 3-grams for all namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      478
0.500000 1.000000            2            2.0        4        1      199
0.750000 1.000000            4            4.0        7        1      259
0.750000 0.750000            8            8.0        7        1      280
0.750000 0.750000           16           16.0        7        7      622
0.781250 0.812500           32           32.0        7        2      517
0.750000 0.718750           64           64.0        3        3      607
0.648438 0.546875          128          128.0        1        7       82
0.628906 0.609375          

For ngram:3 and pass:1 - score:0.9091022803547189


Generating 3-grams for all namespaces.
final_regressor = model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      478
0.500000 1.000000            2            2.0        4        1      199
0.750000 1.000000            4            4.0        7        1      259
0.750000 0.750000            8            8.0        7        1      280
0.750000 0.750000           16           16.0        2        7      472
0.718750 0.687500           32           32.0        1        7     1207
0.687500 0.656250           64           64.0        7        7      304
0.640625 0.593750          128          128.0        5        1      823
0.6

For ngram:3 and pass:3 - score:0.906454329338395


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [ ]:
''' ВАШ КОД ЗДЕСЬ '''

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)